In [ ]:
#coding: utf-8
import numpy as np
import pandas as pd
import re
import jieba
import os
import time
import jieba  #处理中文
import numpy as np
from progressbar import *
start1 =time.clock()

In [ ]:
def obatin_news_from_csv(path_csv):
    old_text=pd.read_csv(path_csv)
    drop_index_na=[]
    for i in range(old_text.shape[0]):
        j=old_text['微博正文'][i]
        if ("抱歉" in j or len(j)<40):
            drop_index_na.append(i)
    text=old_text.drop(drop_index_na)
    text=text.reset_index(drop=True)
    return text

import re
def find_title(text):
    text["标题"]=[-99 for i in range(text.shape[0])]
    text["微博正文（去掉标题）"]=[-99 for i in range(text.shape[0])]
    notitle_index=[]
    title_p=re.compile(r".*\【.*\】")
    progress = ProgressBar()
    for i in progress(range(text.shape[0])):
        title_=re.match(title_p,text["微博正文"][i])
        if title_:
            text["标题"][i]=title_[0]
            text["微博正文（去掉标题）"][i]=re.sub(r".*\【.*\】","",text["微博正文"][i])
        else:
            notitle_index.append(i)
        time.sleep(0.1)
    text=text.drop(notitle_index)
    text=text.reset_index(drop=True)
    return text

In [ ]:
#载入词典和停用词
jieba.load_userdict(r".\自定义词典.txt")
stopwords = [line.strip() for line in open(r".\停用词库.txt",encoding='UTF-8').readlines()]
#清理内容
def clean_text_content(text_content,cut_all=False):
    text_content = re.sub(r"。", ".", text_content)
    text_content = re.sub(r"，", ",", text_content)
    text_content = re.sub(r"“", "'", text_content)
    text_content = re.sub(r"”", "'", text_content)
    text_content = re.sub(r"…", ".", text_content)
    text_content = re.sub(r"@", "@", text_content)
    text_content = re.sub(r" ", " ", text_content)
    text_content = re.sub(r"！", "!", text_content)
    text_content = re.sub(r"？", "?", text_content)
    text_content = re.sub(r"：", ":", text_content)
    text_content = re.sub(r"）", ")", text_content)
    text_content = re.sub(r"（", "(", text_content)
    text_content = re.sub(r"(\d+年)*(\d+月)*(\d+[日])", "", text_content) #日期
    text_content = re.sub(r"\d+[年月日天号人名时例名省市区县院]", "", text_content) 
    text_content = re.sub(r"[第]*[零一二三四五六七八九百千万]+[年月日天号人名时例名省市区县院名例周月年]*", "", text_content) 
    text_content = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text_content) #时间
    text_content = re.sub(r"^[-+]?[0-9]+(\.)?[0-9]*$", "", text_content) #数字
    text_content = re.sub(r"@\S*\:+\s*", " ", text_content) #@小央视频
    text_content = re.sub(r"\[.+\]", "", text_content) #[组图共2张]和[加油]
    text_content = re.sub(r"\(.*?\)", "", text_content) #（环球网）
    
    text_content = re.sub(r"\W+\w*(视频)\s+", "", text_content) #小央视频的秒拍视频
    #de这个bug花费我一晚上时间！！因为一开始写\W+\S*(视频)\s+的话，\S会把,.等也匹配进去，就变成从第一个.匹配了。
    #但是\w是"匹配特殊字符，即非字母、非数字、非汉字、非_"，即不会匹配,.
    #以一下几个做测试：
    #"戳视频↓星辰浩瀚，2020，我们共同努力！新华视点的秒拍视频 @新华视点  "
    #"祝你们每天好“星晴”@iPanda熊猫频道 iPanda熊猫频道的微博视频  "
    #"据悉。日本的独居户已超过三分之一。预计在2040年将达到40%。（全球视频大魔王）全球视频大魔王的微博视频    "

    text_content = re.sub(r"#", "", text_content) ##
    text_content = re.sub(r"(http|https)(://t.cn/)[a-zA-Z0-9]+", "", text_content) #网址(微博上的连接都是http://t.cn/.....形式)
    text_content = re.sub(r"转发理由:", "", text_content) 
    text_content = re.sub(r"转发内容:", "", text_content) 
    text_content = re.sub(r"原始用户:.*", "", text_content) 
    
    word_list=jieba.lcut(text_content,cut_all=cut_all)
    word_list_len=len(word_list)
    i=0
    while i<word_list_len:
        j=word_list[i]
        if j in stopwords or not('\u4e00'<=j<='\u9fff') or len(j)<2:
            word_list.pop(i)
            word_list_len=word_list_len-1
        elif (j=="中方" or j=="我国"):
            word_list[i]="中国"
        elif j=="美方":
            word_list[i]="美国"
        elif j=="贸易战":
            word_list[i]="贸易" 
        else:
            i=i+1
    if " " in word_list:
        word_list.remove(" ")
    return word_list#text_content 

In [ ]:
old_text=obatin_news_from_csv("./数据集1-环球时报/环球时报/1974576991.csv")
text_column=["微博正文","发布时间"]

#把2020年的都去掉
len_old_text=old_text.shape[0]
drop_index=[]
old_pattern=re.compile(r"2020(\-)[0-9]+(\-)[0-9]+")
for i in range(0,len_old_text):
    if old_pattern.match(old_text["发布时间"][i]):
        drop_index.append(i)
text=old_text.drop(drop_index)
text=text.reset_index(drop=True)
text.head(2)

text=find_title(text)
text_shape_old=text.shape[0]
print(text_shape_old)
text.head()

In [ ]:
start =time.clock()

len_text=text.shape[0]

text["微博正文(无标题切词后)"]=[-99 for i in range(len_text)]
text["标题(切词后)"]=[-99 for i in range(len_text)]

drop_na=[]

progress = ProgressBar()

for u in progress(range(0,len_text)):
    cleaned_content=clean_text_content(text["微博正文"][u])
    text["微博正文(无标题切词后)"][u]=" ".join(cleaned_content)

    cleaned_title=clean_text_content(text["标题"][u],True)
    text["标题(切词后)"][u]=" ".join(cleaned_title)

    if pd.isnull(text.loc[u]).any():
        drop_na.append(u)
        print(u) 
    time.sleep(0.1)

text=text.drop(drop_na)
text=text.reset_index(drop=True)   

text["微博正文(有标题切词后)"]=[-99 for i in range(len_text)]
progress = ProgressBar()

for u in progress(range(0,len_text)):
    text["微博正文(有标题切词后)"][u]=text["标题(切词后)"][u]+" "+text["微博正文(无标题切词后)"][u]

end = time.clock()

print('Running time: %s Seconds'%(end-start))

In [ ]:
print("text_shape_old",text_shape_old)
print("text_shape_new",text.shape[0])
text.head()

In [ ]:
text_new=text[["发布时间","微博正文","标题","微博正文（去掉标题）","微博正文(无标题切词后)","标题(切词后)","微博正文(有标题切词后)"]]
text_new.head()

In [ ]:
outputpath='./huanqiu_news_with_title_2.csv'
text_new.to_csv(outputpath,sep=',',index=False,header=True)

描述性分析（词云+每个月发的微博数）

In [ ]:
#按照词频画wordcloud
all_words_space=" ".join(('%s' %id for id in text["微博正文(有标题切词后)"]))

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
wordcloud1 = WordCloud(
       # 设置字体，不然会出现口字乱码，文字的路径是电脑的字体一般路径，可以换成别的
       font_path="C:/Windows/Fonts/simfang.ttf",
       background_color="white",
       # mask参数=图片背景，必须要写上，另外有mask参数再设定宽高是无效的
       width=800,
       height=660,
       max_font_size=200).generate(all_words_space)
plt.imshow(wordcloud1.recolor(), interpolation="bilinear")
plt.axis("off")
#保存生成的图片
wordcloud1.to_file('./wordcloud1.jpg')
plt.show()

In [ ]:
#用jieba.analyse自带的抽取特征词，按照tf-idf画词云
import jieba.analyse
tag=jieba.analyse.extract_tags(all_words_space, topK=100, withWeight=True, allowPOS=())
wc_tfidf ={}
for i in tag:
    wc_tfidf[i[0]]=i[1]
    
wordcloud2 = WordCloud(
       # 设置字体，不然会出现口字乱码，文字的路径是电脑的字体一般路径，可以换成别的
       font_path="C:/Windows/Fonts/simfang.ttf",
       background_color="white",
       # mask参数=图片背景，必须要写上，另外有mask参数再设定宽高是无效的
       width=800,
       height=660,
       max_font_size=200).generate_from_frequencies(wc_tfidf)
plt.imshow(wordcloud2.recolor(), interpolation="bilinear")
plt.axis("off")
#保存生成的图片
wordcloud2.to_file('./result2.jpg')
plt.show()

In [ ]:
#全部微博数据-按月份看数量
import re
time_all={}
for i in range(1,13):
    month_text=0
    if len(str(i))==1:
        mon="0"+str(i)
    else:
        mon=str(i)
    month_re='2019-'+mon
    
    time_list=text["发布时间"]
    for j in time_list:
        if month_re in j:
            month_text=month_text+1
    time_all[mon]=month_text
print(time_all)

import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
month_re_list=[]
for i in range(1,13):
    if len(str(i))==1:
        mon="0"+str(i)
    else:
        mon=str(i)
    month_re='2019-'+mon
    month_re_list.append(month_re)
#print("month_re_list",month_re_list)
new_ticks =month_re_list
x = [int(i[0]) for i in time_all.items()]
y = [int(i[1]) for i in time_all.items()]
print("x",x)
print("y",y)
print("month_re_list",month_re_list)
print("sum(y)",sum(y))

plt.figure(figsize=(10,8),dpi=50)
plt.ylim(0,2000)
plt.xticks(rotation = 90)
plt.bar(month_re_list ,y,width=0.8)
for m,n in zip(x ,y):
    plt.text(m-0.7, n+1, '%.0f' % n, ha='right', va= 'bottom',fontsize=11)